# Instance segmentation evaluation

Test a Detector on A Customized Dataset (here, I use the Mask RCNN)


In [ ]:
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.data import MetadataCatalog
from detectron2.utils.visualizer import Visualizer, ColorMode
import matplotlib.pyplot as plt
import cv2
import os
from detectron2.data.datasets import register_coco_instances

## Load the model

In [ ]:
# model path
# SC_FTAL_F2_80%_best
model_checkpoint_path = "/scratch/tjian/PythonProject/deep_pollutant_SSL/checkpoints/train_weights/SSL_SC_FTAL_F2/Train_80%/model_best_89.1765.pth"
model_config_path = "/scratch/tjian/PythonProject/deep_pollutant_SSL/checkpoints/train_weights/SSL_SC_FTAL_F2/Train_80%/config.yaml"

# baseline_FTAL_F2_80%_best
# model_checkpoint_path = "/scratch/tjian/PythonProject/deep_pollutant_SSL/checkpoints/train_weights/SL_F2_2/Train_80%/model_best_91.1003.pth"
# model_config_path = "/scratch/tjian/PythonProject/deep_pollutant_SSL/checkpoints/train_weights/SL_F2_2/Train_80%/config.yaml"

# SW_FTAL_F2_80%_best
# model_checkpoint_path = "/scratch/tjian/PythonProject/deep_pollutant_SSL/checkpoints/train_weights/SSL_SwAV_FTAL_F2_1/Train_80%/model_best_89.6902.pth"
# model_config_path = "/scratch/tjian/PythonProject/deep_pollutant_SSL/checkpoints/train_weights/SSL_SwAV_FTAL_F2_1/Train_80%/config.yaml"

# model_checkpoint_path = "/scratch/tjian/PythonProject/deep_pollutant_SSL/checkpoints/train_weights/SL_F2_2/Train_100%/model_best_74.8527.pth"
# model_config_path = "/scratch/tjian/PythonProject/deep_pollutant_SSL/checkpoints/train_weights/SL_F2_2/Train_100%/config.yaml"


cfg = get_cfg()
cfg.merge_from_file(model_config_path)
cfg.MODEL.WEIGHTS = model_checkpoint_path 
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
predictor = DefaultPredictor(cfg)


## Register my custom test dataset

dataset should be in COCO format

In [ ]:
# Register Pollutant_test dataset
register_coco_instances("Pollutant_test", {}, "/scratch/tjian/Data/Peng_SSL/SL_test/annotations/test.json", "/scratch/tjian/Data/Peng_SSL/SL_test/test/")
Test_Dataset_name="Pollutant_test"
Class_name=["entrap bean", "free bean"]
# Class_name=["entrapped particle", "free particle"]


## Prediction

### (1) Predict one image

In [ ]:
im = cv2.imread("XXX.jpg")
outputs = predictor(im)
# print(outputs) # It will print num_instances, box location, scores, prediction labels

v = Visualizer(im,
               MetadataCatalog.get(Test_Dataset_name).set(thing_classes=Class_name),
               instance_mode=ColorMode.SEGMENTATION,
               scale=1.0
               )

out = v.draw_instance_predictions(outputs["instances"].to("cpu"))  # do not write "gpu"
img=out.get_image()
result=cv2.imwrite(r"XXX/predict.jpg", img)


### (2) Predict images in a folder

In [ ]:
import os

# define the folder path 
folder_path = r"/scratch/tjian/Data/Peng_SSL/IV-raw/"


# define output folder of predicted images
out_path = r"/scratch/tjian/Data/Peng_SSL/IV-raw_pred/"
# out_path = r"/scratch/tjian/Data/Pollutant_SSL/Predicted/SW_FTAL_F2_80%_best/"

file_names = os.listdir(folder_path)
for filename in file_names:
        if filename.endswith('.jpg'):
            image_path = os.path.join(folder_path, filename)
            out_file_name= os.path.join(out_path, filename)
            im = cv2.imread(image_path)
            # start predicting
            outputs = predictor(im)
            print(outputs)
            print(len(outputs["instances"].get("pred_boxes")))
            v = Visualizer(im,
                           MetadataCatalog.get(Test_Dataset_name).set(thing_classes=Class_name),
                           instance_mode=ColorMode.SEGMENTATION,
                           scale=1.0
                           )
            out = v.draw_instance_predictions(outputs["instances"].to("cpu"))  # do not write "gpu"
            img=out.get_image()
            result=cv2.imwrite(out_file_name, img)

print("Done")


### (3) Test on a test dataset with labels

Run a model to predict images with ground-truth labels and output the accuracy, e.g., AP50

Note: the default output of metric per class is AP@50:5:95. 

If you want to output AP50 the metric per class, you need to change the code "precision = precisions[:, :, idx, 0, -1]" to "precision = precisions[0, :, idx, 0, -1]" in detectron2/evaluation/coco_evaluation.py (Line 358)

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

# define the test dataset path

save_evaluate_file_dir = "/scratch/tjian/PythonProject/deep_pollutant_SSL/checkpoints/train_weights/SSL_SC_FTAL_F2_2/Train_40%/test_results/"

# create the "save_evaluate_file_dir" if they do not exist
if not os.path.isdir(save_evaluate_file_dir):
    os.mkdir(save_evaluate_file_dir)

evaluator = COCOEvaluator(Test_Dataset_name, output_dir=save_evaluate_file_dir)
val_loader = build_detection_test_loader(cfg, Test_Dataset_name)
print(inference_on_dataset(predictor.model, val_loader, evaluator))

### (4) Output accuracy according to prediction and ground-truth files

Compare predictions file (json file) and ground-truth files (json file), and output the accuracy, e.g., AP50

In [ ]:
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

# Load ground truth annotations
coco_gt = COCO('XXX/test.json')

# Load detection results
coco_dt = coco_gt.loadRes('/scratch/tjian/Data/XXX/test_1/test_PR/SSL_pred_results_NMS_0.1/coco_instances_results.json')

# Create COCOEval object
coco_eval = COCOeval(coco_gt, coco_dt, 'bbox')

# Run evaluation
coco_eval.evaluate()
coco_eval.accumulate()
coco_eval.summarize()

# Get precision and recall values at different IoU thresholds
precision = coco_eval.eval['precision']
recall = coco_eval.eval['recall']

### (5) Output confusion matrix on test dataset with annotations, i.e., TP, FP, FN

See the "main_Confusion_matrix_OD.ipynb", but it is not used in this work


### (6) Calculate the FP on a dataset without particles

In [ ]:
import os

# define the folder path 
folder_path = r"XXX"

file_names = os.listdir(folder_path)
FP = 0
for filename in file_names:
        if filename.endswith('.jpg'):
            image_path = os.path.join(folder_path, filename)
            im = cv2.imread(image_path)
            # start predicting
            outputs = predictor(im)
            # print(outputs)
            # print(len(outputs["instances"].get("pred_boxes")))
            FP = FP + len(outputs["instances"].get("pred_boxes"))

print("FP (include free and entrapped particles): ", FP)


### (7) Predict images in a folder, and output the bbox and mask in an excel file

Note: this script works in Linux system

In [ ]:
import os
import pandas as pd

# define the folder path 
folder_path = r"/scratch/tjian/Data/Peng_SSL/IV-count/"

# define output folder of predicted images
out_path = r"/scratch/tjian/Data/Peng_SSL/IV-count_pred/"

# define the path of excel file
excel_file_path = r"/scratch/tjian/Data/Peng_SSL/IV-count_pred.xlsx"

# Initialize a list to hold the data for each instance
data = []

file_names = os.listdir(folder_path)
for filename in file_names:
        if filename.endswith('.jpg'):
            image_path = os.path.join(folder_path, filename)
            out_file_name= os.path.join(out_path, filename)
            im = cv2.imread(image_path)
            # start predicting
            outputs = predictor(im)
            # print(outputs)
            # print("#######################################")
            
            # Extract the instances
            instances = outputs['instances']
            
            # Extract bounding boxes (x1, y1, x2, y2) from 'pred_boxes'
            pred_boxes = instances.pred_boxes.tensor.cpu().numpy()
            # print("Bounding Boxes:", pred_boxes)
            
            # Extract scores from 'scores'
            scores = instances.scores.cpu().numpy()
            # print("scores:", scores)
            
            # Extract classes from 'pred_classes'
            pred_classes = instances.pred_classes.cpu().numpy()
            # print("classes:", pred_classes)
            
            # Extract masks and calculate mask areas from 'pred_masks'
            pred_masks = instances.pred_masks
            mask_areas = pred_masks.sum(dim=(1, 2)).cpu().numpy()  # Sum over height and width dimensions
            # print("Mask Areas:", mask_areas)
            
            # Loop through each instance and get the bounding box and area
            for i in range(len(outputs["instances"].get("pred_boxes"))):
                class_id = pred_classes[i]
                score = scores[i]
                bbox = pred_boxes[i]
                mask_area = mask_areas[i]
                bbox_area= (bbox[2]-bbox[0])*(bbox[3]-bbox[1])
                print(f"Instance {i+1}: Class = {class_id}, Score = {score:.4f}, BBox = {bbox}, bbox_area = {bbox_area}, Area = {mask_area}")
                data.append([filename, class_id, score, bbox[0], bbox[1], bbox[2], bbox[3], bbox_area, mask_area])
            
            # output predicted images
            v = Visualizer(im,
                           MetadataCatalog.get(Test_Dataset_name).set(thing_classes=Class_name),
                           instance_mode=ColorMode.SEGMENTATION,
                           scale=1.0
                           )
            out = v.draw_instance_predictions(outputs["instances"].to("cpu"))  # do not write "gpu"
            img=out.get_image()
            result=cv2.imwrite(out_file_name, img)
            print("Output predicted image: ", filename)

# Create a DataFrame from the data list
df = pd.DataFrame(data, columns=['Image name', 'Class', 'Score', 'BBox_x1', 'BBox_y1', 'BBox_x2', 'BBox_y2', 'Bbox area', 'Mask Area'])

# Save the DataFrame to an Excel file
df.to_excel(excel_file_path, index=False)

print("Data has been saved to", excel_file_path)
